## Resumen ejecutivo


En este trabajo comparamos Trifacta (comercial, Designer Cloud) y pandas (open source) para el manejo de dataframes en tareas típicas de limpieza y preparación de datos. Evaluamos cómo cada herramienta carga, muestra y transforma tablas (tipos de datos, manejo de nulos, joins y agregaciones), junto con criterios de facilidad de uso, coste, acceso y reproducibilidad.

Trifacta ofrece una interfaz gráfica con perfilado y sugerencias que acelera la limpieza sin escribir código y facilita publicar resultados a servicios en la nube; su principal límite es la necesidad de licencia y el acceso a la plataforma. Pandas requiere programar en Python, pero da control total sobre las transformaciones, es gratuito, corre en notebooks y tiene mucho material para aprender, lo que fortalece habilidades técnicas clave.

Conclusión principal: para un estudiante de pregrado, la opción más conveniente para tareas y proyectos es pandas, porque no requiere licencias, funciona en el computador personal y ayuda a practicar Python. Trifacta puede usarse como complemento cuando la universidad provee acceso y se busca prototipado rápido con GUI para presentar resultados.

## 1.1 Selección de Herramientas — Comercial: Trifacta

- Ítem: Data Wrangling / ETL self‑service para analítica
- Comercial (nombre + versión): Alteryx Designer Cloud (Trifacta), 2024.x
- Justificación breve (casos de uso, coste, integración):
  - Trifacta destaca por su interfaz GUI con perfilado automático y sugerencias inteligentes de transformaciones; acelera limpieza/uniones para equipos no técnicos.
  - Integraciones nativas con nubes (BigQuery, Snowflake, S3, etc.) y orquestación; gestión de credenciales y gobernanza.
  - Coste por licencia/consumo; ahorro de tiempo y menor curva de aprendizaje.
- Ejemplo(s) de terceros usados (enlace — autor/organización — fecha — qué parte se usa):
  - “Designer Cloud powered by Trifacta — Quickstart” — Alteryx — consultado nov‑2025 — Se usa el flujo de ejemplo de importar, perfilar, limpiar y publicar para documentar pasos y outputs.


## 1.2 Selección de Herramientas — Open Source: pandas

- Ítem: Manipulación de datos/tabulares en código (dataframes)
- Open Source (nombre + versión): pandas 2.2.x
- Justificación breve (casos de uso, coste, integración):
  - Estándar de facto en ciencia de datos Python; rico ecosistema (NumPy, Matplotlib/Seaborn, scikit‑learn).
  - Gratuito (licencia BSD), fácil de integrar en notebooks, scripts y pipelines (Airflow, Prefect, Dagster).
  - Mayor flexibilidad y control fino vía código; mayor curva de aprendizaje para perfiles no técnicos.
- Ejemplo(s) de terceros usados (enlace — autor/organización — fecha — qué parte se usa):
  - “10 Minutes to pandas” — pandas Development Team — consultado nov‑2025 — Se reutilizan patrones de lectura/limpieza/agrupación para el ejemplo reproducible.


## 2.1 Ejemplo de Uso: Herramienta Comercial — Trifacta (Designer Cloud)

- Fuente del ejemplo: “Designer Cloud powered by Trifacta — Quickstart: Import, Clean, Join and Publish” — Alteryx — consultado nov‑2025. URL de referencia: https://docs.trifacta.com/dataprep/en/quickstart-for-dataprep-by-trifacta.html

- ¿Qué hace el ejemplo y qué resultados produce?
  - Importa un archivo CSV (ventas.csv) y una tabla maestra (productos.csv).
  - Perfila automáticamente las columnas (tipos, valores faltantes, outliers).
  - Aplica transformaciones sugeridas: cambio de tipo, recorte de espacios, división de columnas, reemplazo de nulos, derivación de campos.
  - Realiza un Join por `product_id` y un Group By por `category` con suma de `revenue`.
  - Publica el resultado en formato Parquet y CSV hacia un bucket S3/BigQuery/Snowflake (según conector).
  - Outputs: receta de pasos (auditable), dataset limpio/agregado y reporte de calidad por columna.

- Pasos de reproducción (resumen):
  1) Crear un Flow nuevo y añadir datasets (CSV/Parquet o conectores a nube).
  2) Abrir en Grid; revisar el Visual Profiler y aceptar/editar sugerencias.
  3) Añadir pasos: `Trim`, `Set type`, `Replace missing`, `Split`, `Join` y `Group by`.
  4) Validar resultados en la previsualización, etiquetar tipos/dominios si aplica.
  5) Configurar una salida (Publish) a destino y formato; ejecutar Job (opcional: programar).

- Observaciones (facilidad de uso, limitaciones y coste):
  - Interfaz muy rápida para prototipar; historial de pasos y linaje claros.
  - Requiere licencia/credenciales; límites de fila/tiempo dependen del plan.
  - Ventaja fuerte en conectores gestionados y gobernanza (roles, historiales, programaciones).


## 2.2 Ejemplo de Uso: Herramienta Open Source — pandas

- Fuente del ejemplo: “10 Minutes to pandas” y “User Guide” — pandas Development Team — consultado nov‑2025 — https://pandas.pydata.org/

- ¿Qué hace el ejemplo y qué outputs genera?
  - Crea dos DataFrames de ejemplo en memoria (sin usar archivos CSV), limpia tipos/nulos, hace un `merge` por `product_id`, calcula métricas por categoría y muestra el resultado.
  - Outputs: `DataFrame` limpio y tabla agregada listos para visualizar en el notebook.

- Instrucciones de reproducción (código Python, sin archivos adicionales):

```python
import pandas as pd

# 1) Datos de ejemplo en memoria (sin archivos)
df_sales = pd.DataFrame({
    'order_id':   [1, 2, 3, 4],
    'product_id': [100, 100, 200, 300],
    'date':       ['2025-01-03', '2025-01-05', None, '2025-02-10'],
    'revenue':    [120.5, None, 50, 99.9],
})

df_prod = pd.DataFrame({
    'product_id': [100, 200, 300],
    'category':   ['A', 'B', 'A'],
})

# 2) Limpieza básica
# estandarizar nombres de columnas
df_sales.columns = [c.strip().lower() for c in df_sales.columns]
# tipos
df_sales['date'] = pd.to_datetime(df_sales['date'], errors='coerce')
# nulos
df_sales['revenue'] = df_sales['revenue'].fillna(0.0)

# 3) Join
df = df_sales.merge(df_prod, on='product_id', how='left')

# 4) Agregación
agg = (
    df.groupby('category', dropna=False)
      .agg(total_revenue=('revenue','sum'),
           orders=('order_id','nunique'))
      .reset_index()
      .sort_values('total_revenue', ascending=False)
)

# 5) Visualizar resultados (sin escribir a disco)
print('Tabla agregada por categoría:')
print(agg)
```

- Dependencias:
  - Requiere Python 3.9+ y `pandas>=2.2`. (No se necesitan librerías extra porque no se escriben archivos.)


## 3. Comparación de las Herramientas

Criterios técnicos y prácticos considerados entre Trifacta (comercial) y pandas (open source):

| Criterio | Comercial: Trifacta | Open Source: pandas | Observaciones | Recomendación |
|---|---|---|---|---|
| Facilidad de prototipado | Muy alta (GUI, sugerencias, profiler) | Alta si conoces Python; media para principiantes | Trifacta acelera para no‑desarrolladores | Trifacta para prototipos con negocio; pandas para equipos técnicos |
| Escalabilidad | Gestionada vía conectores/compute cloud | Depende del despliegue (scripts, Dask/Spark) | Pandas en memoria; para big data usar Dask/Spark | Comercial si se requiere gobernanza/SLAs; OSS con Dask/Spark si hay ingeniería |
| Coste | Licencia/consumo y administración | Gratis (licencia BSD); coste de infra/soporte | Trifacta ahorra tiempo; OSS demanda talento | Según presupuesto y equipo |
| Integraciones | Nativas con BigQuery/Snowflake/S3, programaciones | APIs y librerías Python; conectores vía SDK | Comerciales simplifican auth y gobernanza | Si ya hay ecosistema cloud, Trifacta reduce fricción |
| Curva de aprendizaje | Baja-media (no code/low code) | Media-alta (Python) | Documentación amplia en ambos | Elegir según perfil del equipo |
| Trazabilidad/linaje | Recetas visuales, versionado y auditoría | Repositorio de código, tests y CI/CD | En OSS hay que construir gobernanza | Regulado/empresarial: Trifacta; investigación: pandas |

Preguntas guía (respuestas breves):
- ¿Más adecuada para prototipado rápido? Trifacta. ¿Y para producción? Depende: Trifacta para pipelines gobernados; pandas si se integra en un stack de datos ya orquestado.
- ¿Implicancias de coste o gobernanza? Trifacta añade coste pero reduce riesgos y tiempo; pandas requiere disciplina (tests, code review, seguridad) y tiempo de ingeniería.
- ¿Limitaciones observadas? Trifacta depende de la licencia y límites del plan; pandas opera en memoria y requiere librerías extra para big data.


## 4. Conclusiones y Reflexión

- Ventajas y desventajas principales
  - Trifacta: (+) GUI veloz, conectores y gobernanza; (-) coste/licencia, menor flexibilidad que código puro.
  - pandas: (+) Flexibilidad total, sin coste de licencia, ecosistema enorme; (-) requiere conocimientos de programación y gestión de rendimiento.

- ¿Cuándo usar cada una?
  - Comercial (Trifacta): equipos de negocio o mixtos, necesidad de auditar/operar flujos gobernados y conectores gestionados.
  - Open Source (pandas): equipos técnicos que ya usan Python/notebooks y disponen de orquestación (Airflow/Prefect) y control de versiones/CI.

- Recomendaciones por tamaño de empresa
  - Pequeña: comenzar con pandas para minimizar coste; evaluar Trifacta si la colaboración no técnica es clave.
  - Grande/regulada: Trifacta para acelerar y gobernar; complementar con pandas para análisis avanzados y prototipos técnicos.

- Aprendizajes y mejoras futuras
  - Documentar recetas/flows junto con código de referencia (mismas reglas de negocio en ambos).
  - Añadir pruebas automáticas sobre datos (validaciones con Great Expectations/pytest) y métricas de calidad.
  - Extender comparación con datasets más grandes (pandas + Dask/Spark vs Trifacta en cloud compute).
